In [1]:
#take boxes determined by marisol from kathleens biforcation data and calculate average environmental information for them
#want to calculate average SST, SSS, u^2+v^2, and var(u^2+v^2)
import xarray as xr
#import pandas as pd
#import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
import numpy as np
#from math import cos, radians
import datetime as dt

dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
dir_cmc_clim='F:/data/sst/cmc/CMC0.2deg/v2/climatology/'
dir_clim='F:/data/sat_data/oscar/L4/oscar_third_deg/climatology/'
dir_data = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/'

boxes_names = ['NPC','NPC_biforcation','CalCur','Alaska1','Alaska2']
boxes_lon_min = [-160,-130,-125,-130,-148]
boxes_lon_max = [-170,-140,-135,-140,-158]
boxes_lat_min = [41,46,35,52,52]
boxes_lat_max = [46,51,40,57,57]
for i in range(0,5):
    boxes_lon_min[i] += 360
    boxes_lon_max[i] += 360


In [2]:
filename_oscar_clim = dir_clim + 'oscar_v2009_1993_2016_climatology_12202118.nc'
filename_oscar_out = 'F:/data/eddy/bifurcation_ms/oscar_boxes_timeseries.nc'
#make oscar timeseries
#for ibox in range(0,2): #5):
inc=-1
for box in boxes_names:
    inc+=1 
    for lyr in range(2000,2002): #2019):
        filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc.gz'
        ds=xr.open_dataset(filename,drop_variables=['um','vm'])
        ds_subset = ds.sel(longitude=slice(boxes_lon_max[inc],boxes_lon_min[inc]),latitude=slice(boxes_lat_max[inc],boxes_lat_min[inc])) 
        ds_subset['tot_vel']=(ds_subset.u**2+ds_subset.v**2)**.5       
        ds_subset['tot_vel_var']=ds_subset.tot_val**2      
        ds_subset['u_sq']=ds_subset.u**2       
        ds_subset['v_sq']=ds_subset.v**2      
        ds_mean = ds_subset.groupby('time.month').mean('latitude').mean('longitude').resample(time='1MS').mean()    
        if lyr==2000:
            ds_box = ds_mean
        else:
            ds_box = xr.concat([ds_box,ds_mean],dim='time')
        #print(box)
    if box=='NPC':
        ds_newbox = ds_box
    else:
        ds_newbox = xr.concat([ds_newbox,ds_box],dim='box')
        #print(ds_newbox.box)
ds_newbox.to_netcdf(filename_oscar_out)

<xarray.DataArray 'box' (box: 2)>
array([0, 1], dtype=int64)
Dimensions without coordinates: box
<xarray.DataArray 'box' (box: 3)>
array([0, 1, 2], dtype=int64)
Dimensions without coordinates: box
<xarray.DataArray 'box' (box: 4)>
array([0, 1, 2, 3], dtype=int64)
Dimensions without coordinates: box
<xarray.DataArray 'box' (box: 5)>
array([0, 1, 2, 3, 4], dtype=int64)
Dimensions without coordinates: box


In [ ]:
filename_sst_out = 'F:/data/eddy/bifurcation_ms/sst_boxes_timeseries.nc'
inc=-1
for box in boxes_names:
    inc+=1 
    for idyjl in range(1,366):
        clim_sum=np.zeros((1,901,1800))
        clim_cnt=np.zeros((1,901,1800))
        for lyr in range(1992,2017):
        d = datetime.date(lyr,1,1) + datetime.timedelta(idyjl - 1)
        syr=str(d.year).zfill(4)
        smon=str(d.month).zfill(2)
        sdym=str(d.day).zfill(2)
        sjdy=str(idyjl).zfill(3)
        fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
        cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
        nc_fid = Dataset(cmc_filename, 'r')
        sst = nc_fid.variables['analysed_sst'][:]
        if lyr==1992:
            print(d.year, d.month, d.day, idyjl)
            lons = nc_fid.variables['lon'][:]
            lats = nc_fid.variables['lat'][:]
        nc_fid.close()

        
        filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc.gz'
        ds=xr.open_dataset(filename,drop_variables=['um','vm'])
        ds_subset = ds.sel(longitude=slice(boxes_lon_max[inc],boxes_lon_min[inc]),latitude=slice(boxes_lat_max[inc],boxes_lat_min[inc])) 
        ds_subset['tot_vel']=(ds_subset.u**2+ds_subset.v**2)**.5       
        ds_subset['tot_vel_var']=ds_subset.tot_val**2      
        ds_subset['u_sq']=ds_subset.u**2       
        ds_subset['v_sq']=ds_subset.v**2      
        ds_mean = ds_subset.groupby('time.month').mean('latitude').mean('longitude').resample(time='1MS').mean()    
        if lyr==2000:
            ds_box = ds_mean
        else:
            ds_box = xr.concat([ds_box,ds_mean],dim='time')
        #print(box)
    if box=='NPC':
        ds_newbox = ds_box
    else:
        ds_newbox = xr.concat([ds_newbox,ds_box],dim='box')
        #print(ds_newbox.box)
ds_newbox.to_netcdf(filename_oscar_out)